<a href="https://colab.research.google.com/github/akashpambhar21/CE005_ML_AkashPambhar/blob/main/Ml_Lab12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torchvision
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import tensorflow

from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.datasets import mnist
(features_train, targets_train), (features_test,targets_test) = mnist.load_data()
batch_size_train = 64
batch_size_test = 1000
n_iters = 5000
log_interval = 10

num_epochs = n_iters / (len(features_train) / batch_size)
num_epochs = int(num_epochs)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.MNIST('/Lab12', download=True, train=True, transform=transform)
valset = datasets.MNIST('/Lab12', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

In [ ]:
import torch.nn.functional as F

class ANNModel(nn.Module):

  def __init__(self, input_dim, hidden_dim, output_dim):
      super(ANNModel, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(9216, 128)
      self.fc2 = nn.Linear(128, 10)

    # x represents our data
  def forward(self, x):
      # Pass data through conv1
      x = self.conv1(x)
      # Use the rectified-linear activation function over x
      x = F.relu(x)

      x = self.conv2(x)
      x = F.relu(x)

      # Run max pooling over x
      x = F.max_pool2d(x, 2)
      # Pass data through dropout1
      x = self.dropout1(x)
      # Flatten x with start_dim=1
      x = torch.flatten(x, 1)
      # Pass data through fc1
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      # Apply softmax to x
      output = F.log_softmax(x, dim=1)
      return output


  # instantiate ANN
input_dim = 28*28
hidden_dim = 150 
output_dim = 10

# Create ANN
model = ANNModel(input_dim, hidden_dim, output_dim)

# SGD Optimizer
learning_rate = 0.02
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

In [ ]:
from time import time

optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)